In [29]:
import os

os.environ['OMP_NUM_THREADS'] = '2'
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

np.random.seed(42)


def generate_train_and_test(split=0.8, shuffle=True):
    iris = datasets.load_iris()
    data = iris.data[:100, :].astype(np.float32)
    data = preprocessing.minmax_scale(data) * 2 - 1
    label = np.zeros(100).astype(int)
    label[50:] = 1
    return train_test_split(data, label, train_size=split, shuffle=shuffle)


train_x, test_x, train_y, test_y = generate_train_and_test(split=0.2)
print('train sample and feature shape: ', train_x.shape)
print('first train sample: ', train_x[0, :], ' with label: ', train_y[0])

train sample and feature shape:  (20, 4)
first train sample:  [ 0.33333302 -0.24999988  0.804878    0.5294118 ]  with label:  1


In [32]:
from mindquantum import H, RZ, RX, X, Circuit, RY, UN


def encoder(n):
    c = Circuit()
    #create your new encoder here
    for i in range(n):
        c += RZ(f'x{i}').on(i)
    return c


enc = encoder(4).no_grad()
print("encoder circuit:")
print(enc)
print("\n")

def ansatz(n_qubits):
    c = Circuit()
    # create your new ansatz here:
    for i in range(n_qubits):
        c += RX(f"gamma{i}").on(i)
    return c


ans = ansatz(4)
print("ansatz circuit:")
print(ans)
if len(ans) > 15:
    raise ValueError(
        "Too many gates in the ansatz circuit, we limited it to 15 gate.")
tot = enc + ans
print("\ntotal circuit:")
print(tot)

TypeError: Qubit should be a non negative int, but get <class 'str'>!

In [35]:
from mindquantum.ops import QubitOperator
from mindquantum import Hamiltonian

hams = [Hamiltonian(QubitOperator(f'Z{i}')) for i in [2, 3]]
print(hams)

[1.0 [Z2] , 1.0 [Z3] ]


In [37]:
from mindquantum import Simulator
sim = Simulator('projectq', tot.n_qubits)
f_g_ops = sim.get_expectation_with_grad(hams, tot,encoder_params_name=enc.params_name,ansatz_params_name=ans.params_name, parallel_worker=4)

In [39]:
p0 = np.random.uniform(size=len(ans.params_name))
f_g_ops(train_x[:1], p0)

(array([[0.51563151+0.j, 0.71300384+0.j]]),
 array([[[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]]]),
 array([[[-1.16551734e-17+0.j,  3.26886955e-17+0.j, -4.63302540e-01+0.j,
           2.77555756e-17+0.j],
         [ 0.00000000e+00+0.j, -2.77555756e-17+0.j,  6.93889390e-17+0.j,
          -4.86389267e-01+0.j]]]))

In [41]:
import mindspore as ms
from mindquantum.nn import MQLayer
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(42)
net = MQLayer(f_g_ops)

In [42]:
from mindspore import Tensor
print('QNN output: \n', net(Tensor(train_x[:4])))
print('\nweight: \n', net.weight.asnumpy())

QNN output: 
 [[0.69319135 0.8630217 ]
 [0.7600994  0.7977839 ]
 [0.69319135 0.5402506 ]
 [0.7600994  0.63527507]]

weight: 
 [ 0.00495268 -0.00754737  0.00474223 -0.01383794]


In [43]:
import mindspore.dataset as ds
ms.set_seed(42)

batch = 5
train_loader = ds.NumpySlicesDataset({
    'feats': train_x,
    'labs': train_y
},
                                     shuffle=False).batch(batch)
test_loader = ds.NumpySlicesDataset({
    'feats': test_x,
    'labs': test_y
}).batch(batch)

In [44]:
loss = ms.nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
opti = ms.nn.Adam(net.trainable_params(), learning_rate=1e-1)
monitor = ms.train.callback.LossMonitor(16)

In [45]:
model = ms.Model(net, loss, opti, metrics={'Acc': ms.nn.Accuracy()})
model.train(50, train_loader, callbacks=[monitor], dataset_sink_mode=False)

epoch: 4 step: 4, loss is 0.6795387
epoch: 8 step: 4, loss is 0.60958993
epoch: 12 step: 4, loss is 0.5317725
epoch: 16 step: 4, loss is 0.5312529
epoch: 20 step: 4, loss is 0.54098773
epoch: 24 step: 4, loss is 0.54181224
epoch: 28 step: 4, loss is 0.5401618
epoch: 32 step: 4, loss is 0.53966016
epoch: 36 step: 4, loss is 0.5398261
epoch: 40 step: 4, loss is 0.539936
epoch: 44 step: 4, loss is 0.5399176
epoch: 48 step: 4, loss is 0.5398818


In [46]:
predict = np.argmax(ms.ops.Softmax()(model.predict(ms.Tensor(test_x))), axis=1)
corr = model.eval(test_loader, dataset_sink_mode=False)
print(corr)

{'Acc': 0.475}
